In [ ]:
!pip install -q transformers datasets einops bitsandbytes trl peft

In [ ]:
import ast
import json
import torch
import re
import numpy as np
import pandas as pd
from trl import SFTTrainer
from datasets import load_dataset
from datasets import Dataset as d
from torch.utils.data import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, DefaultDataCollator

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-1_5",
    trust_remote_code=True,
    torch_dtype="auto",
    device_map = "cuda"
)

tokenizer = AutoTokenizer.from_pretrained(
    "microsoft/phi-1_5",
    trust_remote_code=True,
    torch_dtype="auto"
)

In [ ]:
with open("/content/drive/MyDrive/datasets/kotlin/dataset.json", "r") as f:
    df = pd.read_json(f)

In [ ]:
df.head()

,input,output
0,interface A,interface B
1,interface B,class Inv<T>(e: T)
2,class Inv<T>(e: T),
3,,"fun <S> intersection(x: Inv<in S>, y: Inv<in S..."
4,"fun <S> intersection(x: Inv<in S>, y: Inv<in S...",


In [ ]:
len(df)

2349307

In [ ]:
train_df = df.sample(frac=0.9256973586768699)
val_df = df.drop(train_df.index)

train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

train_df.head()

,input,output
0,extension.platform.value(,"project.providers.systemProperty(""os.name"")"
1,,if (binary.mode == KotlinJsBinaryMode.DEVELOPM...
2,import org.jetbrains.kotlin.ir.declarations.im...,import org.jetbrains.kotlin.ir.symbols.IrField...
3,},
4,import com.intellij.ide.highlighter.JavaFileType,import com.intellij.openapi.project.Project


In [ ]:
data_template = f""""INPUT_REPLACE_ME

Answer: OUTPUT_REPLACE_ME"""

In [ ]:
train_df["data"] = [data_template.replace("INPUT_REPLACE_ME", train_df.iloc[i]['input']).replace("OUTPUT_REPLACE_ME", train_df.iloc[i]['output'])[1:] for i in range(len(train_df))]
val_df["data"] = [data_template.replace("INPUT_REPLACE_ME", val_df.iloc[i]['input']).replace("OUTPUT_REPLACE_ME", val_df.iloc[i]['output'])[1:] for i in range(len(val_df))]

In [ ]:
del train_df['input']
del train_df['output']

del val_df['input']
del val_df['output']

In [ ]:
import os

os.mkdir('./Outputs')
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
class phiData(Dataset):
    def __init__(self, df):
        self.df = df

    def __getitem__(self, idx):
        return tokenizer.encode(
            self.df.iloc[idx]['data'],
            return_tensors= 'pt',
            max_length= 2048,
            padding = True
        )

    def __len__(self):
        return len(self.df)

train = phiData(train_df)
val = phiData(val_df)

In [ ]:
import transformers

training_args = transformers.TrainingArguments(
    report_to=None,
    auto_find_batch_size=True,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=False,
    save_total_limit=4,
    logging_steps=25,
    output_dir="./Outputs",
    save_strategy='epoch',
    optim="paged_adamw_8bit",
    lr_scheduler_type = 'cosine',
    warmup_ratio = 0.05,
    save_steps= 25
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train,
    eval_dataset=val,
    args=training_args,
    dataset_text_field="data",
    max_seq_length=2048,
    tokenizer=tokenizer,
#     packing=packing,
)

model.config.use_cache = False
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2663: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


RuntimeError: stack expects each tensor to be equal size, but got [1, 38] at entry 0 and [1, 25] at entry 1